

# Import Packages

The main packages that will be used are pandas, tensorflow and torch

In [6]:
import pandas as pd
from datetime import datetime
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score
import random
from torch.utils.data import DataLoader, TensorDataset

# Data processing
We start off by processing the data which is about french trot horse racing into our colab environment. We have already split the data into training and testing data set. We did this through using all data entries after 11-01-2021 as training, and the rest as testing data set.

In [7]:
df = pd.read_csv('training_data.csv', engine='pyarrow')
test_df = pd.read_csv('testing_data.csv', engine='pyarrow')

In [8]:
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', 99999)
df

,,AgeRestriction,Barrier,BeatenMargin,ClassRestriction,CourseIndicator,DamID,Disqualified,Distance,FinishPosition,FoalingCountry,FoalingDate,FrontShoes,Gender,GoingAbbrev,GoingID,HandicapDistance,HandicapType,HindShoes,HorseAge,HorseID,JockeyID,PIRPosition,PriceSP,Prizemoney,RaceGroup,RaceID,RaceOverallTime,RacePrizemoney,RaceStartTime,RacingSubType,Saddlecloth,SexRestriction,SireID,StartType,StartingLine,Surface,TrackID,TrainerID,NoFrontCover,PositionInRunning,WideOffRail,WeightCarried,WetnessScale
0,304,5&6yo,0,999.00,CLM NW$154,,1518963,False,2875.0,BS,FR,2015-03-09,0,M,G,4,0.0,,0,6,1552934,22355,4,24.0,0.0,,1662679,216.68,18000.0,2021-04-22 18:35:00,T,11,,1520456,V,-1,S,951,38144,1,4,2,0.0,3
1,305,6-10yo,1,999.00,NW$325,,1531747,False,2150.0,BS,FR,2013-01-20,0,M,G,4,0.0,,0,8,1538118,27793,0,19.0,0.0,,1662711,153.56,58000.0,2021-04-24 15:17:00,T,1,,1474419,M,1,S,951,37825,-9,-9,-9,0.0,3
2,306,6-10yo,2,6.70,NW$325,,1501267,False,2150.0,5,FR,2013-02-10,0,F,G,4,0.0,,0,8,1539430,22372,2,9.5,2900.0,,1662711,153.56,58000.0,2021-04-24 15:17:00,T,2,,1474389,M,1,S,951,39259,0,2,1,0.0,3
3,307,6-10yo,3,18.80,NW$325,,1516826,False,2150.0,9,FR,2012-04-07,0,M,G,4,0.0,,0,9,1530529,25830,6,142.0,0.0,,1662711,153.56,58000.0,2021-04-24 15:17:00,T,3,,1474471,M,1,S,951,41171,0,6,1,0.0,3
4,308,6-10yo,4,0.40,NW$325,,1545768,False,2150.0,2,SWE,2011-05-26,0,M,G,4,0.0,,0,10,1545769,22395,5,37.0,14500.0,,1662711,153.56,58000.0,2021-04-24 15:17:00,T,4,,1474404,M,1,S,951,37839,0,5,2,0.0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124172,1199656,4yo,0,2.20,NW$10,,1475509,False,2400.0,2,FR,2017-04-12,0,M,G,4,25.0,,2,4,1562776,22617,10,0.0,2750.0,,1666273,190.85,11000.0,2021-07-25 16:07:00,T,13,,1476233,V,-1,T,1023,38048,0,10,2,0.0,3
124173,1199657,4yo,0,2.55,NW$10,,1476776,False,2400.0,3,FR,2017-05-11,2,M,G,4,0.0,,2,4,1562753,22751,7,0.0,1540.0,,1666273,190.85,11000.0,2021-07-25 16:07:00,T,8,,1480826,V,-1,T,1023,39130,0,7,2,0.0,3
124174,1199658,4yo,0,5.80,NW$10,,1520771,False,2400.0,4,FR,2017-04-22,0,M,G,4,0.0,,2,4,1561286,22923,1,0.0,880.0,,1666273,190.85,11000.0,2021-07-25 16:07:00,T,2,,1514152,V,-1,T,1023,38314,1,1,1,0.0,3
124175,1199659,4yo,0,8.00,NW$10,,1506009,False,2400.0,5,FR,2017-03-29,2,F,G,4,0.0,,2,4,1564497,22611,6,0.0,550.0,,1666273,190.85,11000.0,2021-07-25 16:07:00,T,3,,1495119,V,-1,T,1023,38381,0,6,1,0.0,3


The above is a glimpse of the data

We have chosen to drop any columns(not including "FinishPosition", since that is our dependent variable) that have an alphabet in any of the rows. Or in other words, non numeric. This process applies for both training and testing data. We also convert the data entries into numeric where we will be only working with numeric data.

In [9]:
df = df.drop("AgeRestriction", axis=1)
df = df.drop("ClassRestriction", axis=1)
df = df.drop("Disqualified", axis=1)
df = df.drop("FoalingCountry", axis=1)
df = df.drop("FoalingDate", axis=1)
df = df.drop("Gender", axis=1)
df = df.drop("GoingAbbrev", axis=1)
df = df.drop("RaceStartTime", axis=1)
df = df.drop("RacingSubType", axis=1)
df = df.drop("StartType", axis=1)
df = df.drop("CourseIndicator", axis=1)
df = df.drop("Surface", axis=1)
df = df.drop("HandicapType", axis=1)
df = df.drop("RaceGroup", axis=1)
df = df.drop("SexRestriction", axis=1)
df = df.drop("", axis=1)

test_df = test_df.drop("AgeRestriction", axis=1)
test_df = test_df.drop("ClassRestriction", axis=1)
test_df = test_df.drop("Disqualified", axis=1)
test_df = test_df.drop("FoalingCountry", axis=1)
test_df = test_df.drop("FoalingDate", axis=1)
test_df = test_df.drop("Gender", axis=1)
test_df = test_df.drop("GoingAbbrev", axis=1)
test_df = test_df.drop("RaceStartTime", axis=1)
test_df = test_df.drop("RacingSubType", axis=1)
test_df = test_df.drop("StartType", axis=1)
test_df = test_df.drop("CourseIndicator", axis=1)
test_df = test_df.drop("Surface", axis=1)
test_df = test_df.drop("HandicapType", axis=1)
test_df = test_df.drop("RaceGroup", axis=1)
test_df = test_df.drop("SexRestriction", axis=1)
test_df = test_df.drop("", axis=1)

In [10]:
df

,Barrier,BeatenMargin,DamID,Distance,FinishPosition,FrontShoes,GoingID,HandicapDistance,HindShoes,HorseAge,HorseID,JockeyID,PIRPosition,PriceSP,Prizemoney,RaceID,RaceOverallTime,RacePrizemoney,Saddlecloth,SireID,StartingLine,TrackID,TrainerID,NoFrontCover,PositionInRunning,WideOffRail,WeightCarried,WetnessScale
0,0,999.00,1518963,2875.0,BS,0,4,0.0,0,6,1552934,22355,4,24.0,0.0,1662679,216.68,18000.0,11,1520456,-1,951,38144,1,4,2,0.0,3
1,1,999.00,1531747,2150.0,BS,0,4,0.0,0,8,1538118,27793,0,19.0,0.0,1662711,153.56,58000.0,1,1474419,1,951,37825,-9,-9,-9,0.0,3
2,2,6.70,1501267,2150.0,5,0,4,0.0,0,8,1539430,22372,2,9.5,2900.0,1662711,153.56,58000.0,2,1474389,1,951,39259,0,2,1,0.0,3
3,3,18.80,1516826,2150.0,9,0,4,0.0,0,9,1530529,25830,6,142.0,0.0,1662711,153.56,58000.0,3,1474471,1,951,41171,0,6,1,0.0,3
4,4,0.40,1545768,2150.0,2,0,4,0.0,0,10,1545769,22395,5,37.0,14500.0,1662711,153.56,58000.0,4,1474404,1,951,37839,0,5,2,0.0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124172,0,2.20,1475509,2400.0,2,0,4,25.0,2,4,1562776,22617,10,0.0,2750.0,1666273,190.85,11000.0,13,1476233,-1,1023,38048,0,10,2,0.0,3
124173,0,2.55,1476776,2400.0,3,2,4,0.0,2,4,1562753,22751,7,0.0,1540.0,1666273,190.85,11000.0,8,1480826,-1,1023,39130,0,7,2,0.0,3
124174,0,5.80,1520771,2400.0,4,0,4,0.0,2,4,1561286,22923,1,0.0,880.0,1666273,190.85,11000.0,2,1514152,-1,1023,38314,1,1,1,0.0,3
124175,0,8.00,1506009,2400.0,5,2,4,0.0,2,4,1564497,22611,6,0.0,550.0,1666273,190.85,11000.0,3,1495119,-1,1023,38381,0,6,1,0.0,3


In [11]:
df = df.apply(pd.to_numeric, errors='coerce')
test_df = test_df.apply(pd.to_numeric, errors='coerce')

In [12]:
df = df.dropna()
test_df = test_df.dropna()

In [13]:
df = df.astype(float)
test_df = test_df.astype(float)

# Neural Network Using Pytorch

## Model creation: neural network
To predict the "FinishPosition" for each HorseID of each RaceID, we decided to create a neural network model with 8 layers. We will be doing this though creating a function and then applying the function to our training and testing data. Each of the layers are linear regression based.

In [14]:
class HorsePredictor(nn.Module):
    def __init__(self):
        super(HorsePredictor, self).__init__()
        self.fc1 = nn.Linear(27, 64)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(64, 128)
        self.sigmoid = nn.Sigmoid()
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 32)
        self.fc5 = nn.Linear(32, 16)
        self.fc6 = nn.Linear(16, 8)
        self.fc7 = nn.Linear(8, 32)
        self.fc8 = nn.Linear(32, 20)


    def forward(self, x):
        x = self.fc1(x)
        # x = self.relu(x)
        x = self.fc2(x)
        # x = self.sigmoid(x)
        x = self.fc3(x)
        # x = self.relu(x)
        x = self.fc4(x)
        x = self.fc5(x)
        x = self.fc6(x)
        x = self.fc7(x)
        x = self.fc8(x)
        return x


## Prepare the training and test data for model training


### Training data

We decided to isolate for all the unique RaceIDs such that we can use these to create the batchs needed for model training. But note that there are other ways to this. We thought that its best to make predictions based on information from each race.

In [15]:
# Create a dictionary to store separate DataFrames
dfss = []

# Iterate through unique values in the 'Category' column
for race_id in df['RaceID'].unique():
    # Create a new DataFrame for each unique value
    sub_df = df[df['RaceID'] == race_id]
    dfss.append(sub_df)

In [16]:
df_features = []
df_labels = []
for i in range(len(dfss)):
  feature, label = dfss[i].drop('FinishPosition', axis=1), dfss[i]['FinishPosition']
  df_features.append(feature)
  df_labels.append(label)

### Test data

We use the same procedure for the training data here for the testing data.

In [17]:
# Test

# Create a dictionary to store separate DataFrames
test_dfss = []

# Iterate through unique values in the 'Category' column
for race_id in test_df['RaceID'].unique():
    # Create a new DataFrame for each unique value
    sub_df = test_df[test_df['RaceID'] == race_id]
    test_dfss.append(sub_df)


test_df_features = []
test_df_labels = []
for i in range(len(test_dfss)):
  feature, label = test_dfss[i].drop('FinishPosition', axis=1), test_dfss[i]['FinishPosition']
  test_df_features.append(feature)
  test_df_labels.append(label)

### Change to tensor

We develop the tensors needed for the neural networks.

In [18]:
combined = list(zip(df_features, df_labels))
random.shuffle(combined)
df_features[:], df_labels[:] = zip(*combined)

train_feature_tensors = [torch.tensor(df1.values, dtype=torch.float32) for df1 in df_features]
train_label_tensors = [torch.tensor(df1.values, dtype=torch.long) for df1 in df_labels]

test_feature_tensors = [torch.tensor(df1.values, dtype=torch.float32) for df1 in test_df_features]
test_label_tensors = [torch.tensor(df1.values, dtype=torch.long) for df1 in test_df_labels]

## Train the NN model

We first apply the function developed above, and then define the criterion to be cross entropy loss, and use adam optimizer.

In [19]:
# input_size = 27 # Specify the number of features
# hidden_size = 64 # Define the size of the hidden layer
# output_size = 19 # Predicting the probability of first place

model = HorsePredictor()

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005, weight_decay=1e-4)

num_epochs = 20
# epoch = 0

loss_difference = 0.001
loss_1 = 10
loss_2 = 0
# for epoch in range(num_epochs):
for epoch in range(num_epochs):
    # inputs = torch.Tensor(train_feature_tensors).float()
    # labels = torch.Tensor(train_label_tensors).long()
    for features, labels in zip(train_feature_tensors, train_label_tensors):
        # for fea, lab in zip(features, labels):
        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')
    # loss_2 = loss_1
    # loss_1 = loss.item()
    # epoch += 1
    # if loss.item() < 0.5:
    #   break

print('Training finished')

## Test the model

We validate our results on the testing data set.

In [ ]:
model.eval()  # Set the model to evaluation mode
correct = 0
total = 0

with torch.no_grad():  # Disable gradient computation during validation
    for inputs, labels in zip(test_feature_tensors, test_label_tensors):
        outputs = model(inputs)
        predicted = torch.argmax(outputs, 1)
        # predicted = outputs  # Get the predicted class indices

        # Display predicted output and true label
        # for i in range(len(predicted)):
        #     print(f'Predicted: {predicted[i]}, True Label: {labels[i].item()}')

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Validation - Accuracy: {accuracy:.2f}%')

Validation - Accuracy: 69.95%


## Normalization

The goal of this section is to convert the placement of the horses into their win probability. We first normalize the data, and then assign weights to each of the horses within the data, the closer they are to 1st place, the higher the weight. The weights are assigned through dividing the placement of each horse by 1. (Taking the reciprical of their placement). We then divide this by the sum of the total amount of recipricals we have taken. This way we have assigned a quotient to each of the horses where the sum of the quotients will be 1. We then assign this as the probability after normalization and introduce it into a new column.

In [ ]:
model.eval()  # Set the model to evaluation mode
correct = 0
total = 0

lst = []

with torch.no_grad():  # Disable gradient computation during validation
    for inputs, labels in zip(test_feature_tensors, test_label_tensors):
        outputs = model(inputs)
        predicted = torch.argmax(outputs, 1)
        # predicted = outputs  # Get the predicted class indices
        predicted = predicted.numpy()
        lst.append(predicted)


NameError: ignored

In [ ]:
#calculate winprobability
lsst = []
for pre in lst:
    pr = (1 / pre) / np.sum(1 / pre)
    lsst.append(pr.reshape(-1, 1))

NameError: ignored

In [ ]:
test_features_numpy = torch.cat(test_feature_tensors, dim=0).numpy()
predicted_test_labels = np.vstack(lsst)

NameError: ignored

In [ ]:
predicted_test_with_labels = np.concatenate((test_features_numpy, predicted_test_labels), axis=1)

In [ ]:
predicted_test_df = pd.DataFrame(predicted_test_with_labels)

In [ ]:
predicted_test_df.columns = ['Barrier', 'BeatenMargin', 'DamID', 'Distance',
       'FrontShoes', 'GoingID', 'HandicapDistance', 'HindShoes', 'HorseAge',
       'HorseID', 'JockeyID', 'PIRPosition', 'PriceSP', 'Prizemoney', 'RaceID',
       'RaceOverallTime', 'RacePrizemoney', 'Saddlecloth', 'SireID',
       'StartingLine', 'TrackID', 'TrainerID', 'NoFrontCover',
       'PositionInRunning', 'WideOffRail', 'WeightCarried', 'WetnessScale', 'win_probability']

In [ ]:
predicted_test = predicted_test_df[['HorseID', 'RaceID', 'win_probability']]

In [ ]:
predicted_test

# Predict the custom data

In [ ]:
custom_df = pd.read_csv('custom_data.csv', engine='pyarrow')

In [ ]:
custom_df = custom_df.drop("AgeRestriction", axis=1)
custom_df = custom_df.drop("ClassRestriction", axis=1)
custom_df = custom_df.drop("Disqualified", axis=1)
custom_df = custom_df.drop("FoalingCountry", axis=1)
custom_df = custom_df.drop("FoalingDate", axis=1)
custom_df = custom_df.drop("Gender", axis=1)
custom_df = custom_df.drop("GoingAbbrev", axis=1)
custom_df = custom_df.drop("RaceStartTime", axis=1)
custom_df = custom_df.drop("RacingSubType", axis=1)
custom_df = custom_df.drop("StartType", axis=1)
custom_df = custom_df.drop("CourseIndicator", axis=1)
custom_df = custom_df.drop("Surface", axis=1)
custom_df = custom_df.drop("HandicapType", axis=1)
custom_df = custom_df.drop("RaceGroup", axis=1)
custom_df = custom_df.drop("SexRestriction", axis=1)
custom_df = custom_df.drop("", axis=1)

In [ ]:
custom_df = custom_df.apply(pd.to_numeric, errors='coerce')
custom_df = custom_df.dropna()
custom_df = custom_df.astype(float)

In [ ]:
# Create a dictionary to store separate DataFrames
custom_dfss = []

# Iterate through unique values in the 'Category' column
for race_id in custom_df['RaceID'].unique():
    # Create a new DataFrame for each unique value
    sub_df = custom_df[custom_df['RaceID'] == race_id]
    custom_dfss.append(sub_df)

In [ ]:
custom_df_features = []
custom_df_labels = []
for i in range(len(custom_dfss)):
  feature, label = custom_dfss[i].drop('FinishPosition', axis=1), custom_dfss[i]['FinishPosition']
  custom_df_features.append(feature)
  custom_df_labels.append(label)

In [ ]:
custom_feature_tensors = [torch.tensor(df1.values, dtype=torch.float32) for df1 in custom_df_features]
custom_label_tensors = [torch.tensor(df1.values, dtype=torch.long) for df1 in custom_df_labels]

In [ ]:
model.eval()  # Set the model to evaluation mode


custom_lst = []

with torch.no_grad():  # Disable gradient computation during validation
    for inputs, labels in zip(custom_feature_tensors, custom_label_tensors):
        outputs = model(inputs)
        predicted = torch.argmax(outputs, 1)
        # predicted = outputs  # Get the predicted class indices
        predicted = predicted.numpy()
        custom_lst.append(predicted)


In [ ]:
#calculate winprobability
custom_lsst = []
for pre in custom_lst:
    pr = (1 / pre) / np.sum(1 / pre)
    custom_lsst.append(pr.reshape(-1, 1))

In [ ]:
custom_features_numpy = torch.cat(custom_feature_tensors, dim=0).numpy()
predicted_custom_labels = np.vstack(custom_lsst)

In [ ]:
predicted_custom_with_labels = np.concatenate((custom_features_numpy, predicted_custom_labels), axis=1)

In [ ]:
predicted_custom_df = pd.DataFrame(predicted_custom_with_labels)

In [ ]:
predicted_custom_df.columns = ['Barrier', 'BeatenMargin', 'DamID', 'Distance',
       'FrontShoes', 'GoingID', 'HandicapDistance', 'HindShoes', 'HorseAge',
       'HorseID', 'JockeyID', 'PIRPosition', 'PriceSP', 'Prizemoney', 'RaceID',
       'RaceOverallTime', 'RacePrizemoney', 'Saddlecloth', 'SireID',
       'StartingLine', 'TrackID', 'TrainerID', 'NoFrontCover',
       'PositionInRunning', 'WideOffRail', 'WeightCarried', 'WetnessScale', 'win_probability']

In [ ]:
predicted_result = predicted_custom_df[['HorseID', 'RaceID', 'win_probability']]

In [ ]:
predicted_result

,HorseID,RaceID,win_probability
0,1545779.0,1662903.0,0.278330
1,1545897.0,1662903.0,0.139165
2,1547175.0,1662903.0,0.092777
3,1549945.0,1662903.0,0.069583
4,1548420.0,1662903.0,0.055666
...,...,...,...
88184,1563119.0,1666241.0,0.096419
88185,1563782.0,1666241.0,0.385675
88186,1563560.0,1666241.0,0.077135
88187,1564835.0,1666241.0,0.064279


In [ ]:
predicted_result.to_csv("predicted_result.csv", index=False)

In [ ]:
predicted_result.to_parquet('predicted_result.parquet')


NameError: ignored